In [6]:
#!pip install treelib
#!pip install tensorflow
#!pip install -U ckiptagger[tf,gdown]


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [110]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

# 讀取聯合報資料

In [79]:
df = pd.read_csv("udndata社論1991_2020.csv")
df.head()

,date,title,content
0,1991-01-01,民國八十年是政治決定國運年,中華民國今天跨進建國八十年代。這是充滿希望的年代，也是充滿變數的年代。中國在這個年代如何開拓...
1,1991-01-02,在政治年中創造政治奇蹟,今年是政治年，因為李登輝總統正式宣佈將在今年五月終止動員戡亂時期，國民大會臨時會也準備在今年...
2,1991-01-03,融合政黨政治的運作於憲政革新使命的達成上,民國八十年是我國憲政發展最具關鍵的一年。由於我國已經邁向多元的民主化進程，政黨政治的運作成敗...
3,1991-01-04,使八十年代成為同胞族群修好的年代-,走過了紛爭、擾攘、對立、衝突的民國七十年代，展望嶄新的民國八十年代，就社會文化的角度而言，應...
4,1991-01-05,八十年代亟待建立新的發展共識,中華民國步入八十年代便立即面臨憲政改革與經濟復甦的重大問題。就經濟而言，隨著七十年代的流逝，...


In [80]:
# 增加年份以利篩選
df["year"] = df["date"].str.slice(0,4)
df.head()

,date,title,content,year
0,1991-01-01,民國八十年是政治決定國運年,中華民國今天跨進建國八十年代。這是充滿希望的年代，也是充滿變數的年代。中國在這個年代如何開拓...,1991
1,1991-01-02,在政治年中創造政治奇蹟,今年是政治年，因為李登輝總統正式宣佈將在今年五月終止動員戡亂時期，國民大會臨時會也準備在今年...,1991
2,1991-01-03,融合政黨政治的運作於憲政革新使命的達成上,民國八十年是我國憲政發展最具關鍵的一年。由於我國已經邁向多元的民主化進程，政黨政治的運作成敗...,1991
3,1991-01-04,使八十年代成為同胞族群修好的年代-,走過了紛爭、擾攘、對立、衝突的民國七十年代，展望嶄新的民國八十年代，就社會文化的角度而言，應...,1991
4,1991-01-05,八十年代亟待建立新的發展共識,中華民國步入八十年代便立即面臨憲政改革與經濟復甦的重大問題。就經濟而言，隨著七十年代的流逝，...,1991


In [34]:
# 測試資料，嘗試取得2011年前五篇報導內文
list(df[df["year"]=="2011"]['content'])[:5]

['今天是中華民國一百年元旦。這是艱辛困難的一百年，也是繁華豐盛的一百年。我們在此願與國人相互期勉，同心共創中華民國的「第五個奇蹟」：兩岸奇蹟。回首中華民國一百年，至少創造了五大奇蹟，包括：共和奇蹟、再生奇蹟、經濟奇蹟、民主奇蹟，及愈來愈凸顯的兩岸奇蹟。在喧囂的現實世界內，人們陷於眼前的擾攘裡；「不識奇蹟真面目，只緣身在奇蹟中」。值此民國百年元旦，我們願與國人一同檢視這「百年五大奇蹟」，回味中華民國的辛酸苦辣，思考中華民國的風華與信念，並為這「奇蹟之國」寄以勉勵與祝福。一九一二年，中華民國成為亞洲第一個民主共和國；這個「共和奇蹟」的最大成就，是創造了三民主義「民族╱民權╱民生」及「天下為公」的立國精神資產。這使得百年來經歷飄搖離亂的中華民國，今日仍能在台灣及兩岸間操持了正確的文明標竿，而孫中山先生的理想亦仍是兩岸的共同語言及共同追求。一九四九年，中華人民共和國統有大陸，中華民國政府則播遷到台灣；若非金門之役驚險獲勝，且韓戰改變了世局經緯，中華民國極可能已在國際棄絕下覆亡，台灣亦早已併入中華人民共和國。然而，一九四九年以來，世人卻見證了中華民國的「再生奇蹟」。回首前塵，中華民國政府在冷戰中的國際槓桿操作大致正確，與中共政權的對抗如八二三砲戰等亦稱成功，且對內的社經建設也能循序漸進；這使得台灣沒有併入中華人民共和國，因而也沒有經歷三面紅旗、文化大革命等浩劫，且中華民國在六十餘年後能有今日舉世共睹的政經成就，這自是「再生奇蹟」。「再生奇蹟」中的凸顯者是「經濟奇蹟」。台灣幾無自然資源，但自五○代出口蔗糖、樟腦，至當今廿一世紀成為世界資訊產業重鎮，始終在經濟建設上表現不凡。而且，此種經濟建設在近年Ｍ型社會浮現前，經由教育普及及分配管理等，相當均允地回饋給庶民百姓。中華民國的經濟奇蹟曾是世界的範本。一九八八年解嚴後的「民主奇蹟」，則是中華民國在政經表現上的再升級，亦是綜合國力的再提升。其實，中華民國的民主建設，由地方自治始，一直在進行「分期付款式」的民主實踐；正如社經建設，亦是由三七五減租始，循序進階躐等，且以社經建設成果回饋並支撐民主建設。台灣是一個經歷二二八事件及白色恐怖統治的社會，且又統獨分歧、族群多元；但中華民國如今則是由里長至總統皆公民直選的國家，而國家社會在兩岸競合的重大壓力下，在民主政治的喧囂裡，及在政黨輪替的轉換中，迄今仍能維持正確及進取的大方向，自然可稱「民主

# ckiptagger 斷詞 — 五篇

In [8]:
# 載入資料庫
ws = WS("/Users/liuchenli/data")

/opt/anaconda3/lib/python3.7/site-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/opt/anaconda3/lib/python3.7/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:988: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  partitioner=maybe_partitioner)
/opt/anaconda3/lib/python3.7/site-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:996: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  initializer=initializer)
2023-03-05 19:48:00.696709: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (o

In [62]:
# 將測試資料斷詞後放入同一個list中
# result = sum(x, [])
year_content = sum(ws(list(df[df["year"]=="2011"]['content'])[:5]), [])
year_content

['今天',
 '是',
 '中華民國',
 '一百',
 '年',
 '元旦',
 '。',
 '這',
 '是',
 '艱辛',
 '困難',
 '的',
 '一百',
 '年',
 '，',
 '也',
 '是',
 '繁華',
 '豐盛',
 '的',
 '一百',
 '年',
 '。',
 '我們',
 '在',
 '此',
 '願',
 '與',
 '國人',
 '相互',
 '期勉',
 '，',
 '同心',
 '共創',
 '中華民國',
 '的',
 '「',
 '第五',
 '個',
 '奇蹟',
 '」',
 '：',
 '兩岸',
 '奇蹟',
 '。',
 '回首',
 '中華民國',
 '一百',
 '年',
 '，',
 '至少',
 '創造',
 '了',
 '五',
 '大',
 '奇蹟',
 '，',
 '包括',
 '：',
 '共和',
 '奇蹟',
 '、',
 '再生',
 '奇蹟',
 '、',
 '經濟',
 '奇蹟',
 '、',
 '民主',
 '奇蹟',
 '，',
 '及',
 '愈來愈',
 '凸顯',
 '的',
 '兩岸',
 '奇蹟',
 '。',
 '在',
 '喧囂',
 '的',
 '現實',
 '世界',
 '內',
 '，',
 '人們',
 '陷於',
 '眼前',
 '的',
 '擾攘',
 '裡',
 '；',
 '「',
 '不',
 '識',
 '奇蹟',
 '真面目',
 '，',
 '只',
 '緣身',
 '在',
 '奇蹟',
 '中',
 '」',
 '。',
 '值',
 '此',
 '民國',
 '百年',
 '元旦',
 '，',
 '我們',
 '願',
 '與',
 '國人',
 '一同',
 '檢視',
 '這',
 '「',
 '百',
 '年',
 '五',
 '大',
 '奇蹟',
 '」',
 '，',
 '回味',
 '中華民國',
 '的',
 '辛酸苦辣',
 '，',
 '思考',
 '中華民國',
 '的',
 '風華',
 '與',
 '信念',
 '，',
 '並',
 '為',
 '這',
 '「',
 '奇蹟',
 '之',
 '國',
 '」',
 '寄',
 '以',
 '勉勵',
 '與',
 '祝福',
 '。',
 '一九一

In [61]:
# 定義停用詞
with open('stopwords_zhTW.txt') as f:
    stopword_list = [s for s in f.read().split('\n')]

# https://github.com/A-baoYang/product-selection-analysis/blob/main/stopwords_zhTW.txt

In [63]:
# 將斷詞結果中的停用詞去除，並以空格合併成一個長字串
w_string = str()
for w in year_content:
        if w not in stopword_list:
            w_string += w + ' '
w_string

'中華民國 一百 年 元旦 艱辛 困難 一百 年 繁華 豐盛 一百 年 願 國人 相互 期勉 同心 共創 中華民國 第五 奇蹟 兩岸 奇蹟 回首 中華民國 一百 年 至少 創造 奇蹟 包括 共和 奇蹟 再生 奇蹟 經濟 奇蹟 民主 奇蹟 愈來愈 凸顯 兩岸 奇蹟 喧囂 現實 世界 陷於 眼前 擾攘 裡 識 奇蹟 真面目 緣身 奇蹟 中 值 民國 百年 元旦 願 國人 一同 檢視 百 年 奇蹟 回味 中華民國 辛酸苦辣 思考 中華民國 風華 信念 奇蹟 國 寄 勉勵 祝福 一九一二年 中華民國 亞洲 第一 民主 共和國 共和 奇蹟 成就 創造 三民主義 民族 民權 民生 天下為公 立國 精神 資產 百 年 經歷 飄搖離亂 中華民國 今日 台灣 兩岸 間 操持 正確 文明 標竿 孫中山 理想 兩岸 語言 追求 一九四九年 中華 共和國 統有 大陸 中華民國 政府 播遷到 台灣 金門 役 驚險 獲勝 韓戰 改變 世局 經緯 中華民國 國際 棄絕 覆亡 台灣 早已 併入 中華 共和國 。然而 一九四九年 世人 見證 中華民國 再生 奇蹟 回首 前塵 中華民國 政府 冷戰 中 國際 槓桿 操作 正確 中共 政權 對抗 八二三 砲戰 稱 成功 社經 建設 循序漸進 台灣 併入 中華 共和國 經歷 面 紅旗 文化大革命 浩劫 中華民國 六十餘 年 今日 舉世共睹 政經 成就 自是 再生 奇蹟 再生 奇蹟 中 凸顯 經濟 奇蹟 台灣 自然 資源 五○代 出口 蔗糖 樟腦 當今 廿一世紀 世界 資訊 產業 重鎮 始終 經濟 建設 表現 不凡 經濟 建設 近年 Ｍ型 社會 浮現 前 經由 教育 普及 分配 管理 允 回饋給 庶民 百姓 中華民國 經濟 奇蹟 世界 範本 一九八八年 解嚴 民主 奇蹟 中華民國 政經 表現 升級 綜合 國力 提升 中華民國 民主 建設 地方 自治 始 一直 分期付 款式 民主 實踐 正 社經 建設 三七五 減租 始 循序 進階躐 社經 建設 成果 回饋 支撐 民主 建設 台灣 一 經歷 二二八事件 白色恐怖 統治 社會 統獨 分歧 族群 多元 中華民國 里長 總統 皆 公民 直選 國家 國家 社會 兩岸 競合 壓力 民主 政治 喧囂 裡 政黨 輪替 轉換 中 正確 進取 方向 自然 稱 民主 奇蹟 中華民國 第五 奇蹟 延續 六十餘 年 兩岸 奇蹟 前述 

# ckiptagger 斷詞 — 十年各五篇

In [64]:
years_content_ws = []
for i in range(2011,2021):
    year_content = sum(ws(list(df[df["year"]==str(i)]['content'])[:5]), [])
    w_string = str()
    for w in year_content:
        if w not in stopword_list:
            w_string += w + ' '
    years_content_ws.append(w_string)

years_content_ws

['中華民國 一百 年 元旦 艱辛 困難 一百 年 繁華 豐盛 一百 年 願 國人 相互 期勉 同心 共創 中華民國 第五 奇蹟 兩岸 奇蹟 回首 中華民國 一百 年 至少 創造 奇蹟 包括 共和 奇蹟 再生 奇蹟 經濟 奇蹟 民主 奇蹟 愈來愈 凸顯 兩岸 奇蹟 喧囂 現實 世界 陷於 眼前 擾攘 裡 識 奇蹟 真面目 緣身 奇蹟 中 值 民國 百年 元旦 願 國人 一同 檢視 百 年 奇蹟 回味 中華民國 辛酸苦辣 思考 中華民國 風華 信念 奇蹟 國 寄 勉勵 祝福 一九一二年 中華民國 亞洲 第一 民主 共和國 共和 奇蹟 成就 創造 三民主義 民族 民權 民生 天下為公 立國 精神 資產 百 年 經歷 飄搖離亂 中華民國 今日 台灣 兩岸 間 操持 正確 文明 標竿 孫中山 理想 兩岸 語言 追求 一九四九年 中華 共和國 統有 大陸 中華民國 政府 播遷到 台灣 金門 役 驚險 獲勝 韓戰 改變 世局 經緯 中華民國 國際 棄絕 覆亡 台灣 早已 併入 中華 共和國 。然而 一九四九年 世人 見證 中華民國 再生 奇蹟 回首 前塵 中華民國 政府 冷戰 中 國際 槓桿 操作 正確 中共 政權 對抗 八二三 砲戰 稱 成功 社經 建設 循序漸進 台灣 併入 中華 共和國 經歷 面 紅旗 文化大革命 浩劫 中華民國 六十餘 年 今日 舉世共睹 政經 成就 自是 再生 奇蹟 再生 奇蹟 中 凸顯 經濟 奇蹟 台灣 自然 資源 五○代 出口 蔗糖 樟腦 當今 廿一世紀 世界 資訊 產業 重鎮 始終 經濟 建設 表現 不凡 經濟 建設 近年 Ｍ型 社會 浮現 前 經由 教育 普及 分配 管理 允 回饋給 庶民 百姓 中華民國 經濟 奇蹟 世界 範本 一九八八年 解嚴 民主 奇蹟 中華民國 政經 表現 升級 綜合 國力 提升 中華民國 民主 建設 地方 自治 始 一直 分期付 款式 民主 實踐 正 社經 建設 三七五 減租 始 循序 進階躐 社經 建設 成果 回饋 支撐 民主 建設 台灣 一 經歷 二二八事件 白色恐怖 統治 社會 統獨 分歧 族群 多元 中華民國 里長 總統 皆 公民 直選 國家 國家 社會 兩岸 競合 壓力 民主 政治 喧囂 裡 政黨 輪替 轉換 中 正確 進取 方向 自然 稱 民主 奇蹟 中華民國 第五 奇蹟 延續 六十餘 年 兩岸 奇蹟 前述

In [65]:
len(years_content_ws)

10

# 計算TF-IDF

In [77]:
# 10年 vs 所有出現過的字詞
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(years_content_ws)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
result = pd.DataFrame(denselist, columns=feature_names)
result

,app,iphone,line,qe4,uber,voting,〇九,〇八年,一〇五,一一,...,ｉｍｆ,ｉｔａ,ｉｔａ２,ｍ型,ｏｐｅｃ,ｒｃｅｐ,ｔｉｆａ,ｔｐｐ,ｕ型,ｗｔｏ
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.011511,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013909,...,0.011824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.014719,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.012512,0.000000,0.000000,0.000000,0.000000,0.010947,0.014719,0.010947,0.014719,0.014719
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.081324,0.000000,0.081324,0.000000,0.000000
4,0.014159,0.000000,0.000000,0.000000,0.368127,0.014159,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.014159,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.033462,0.033462,0.000000,0.000000,0.012443,0.000000,0.074660,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015080,0.017739,0.017739,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.017493,0.000000,0.000000,0.000000,0.000000,0.014871,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.015662,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [78]:
result.to_csv("tfidf_result.csv", index=False)

# 排序TF-IDF

In [97]:
# 2011年前10名高分的關鍵詞
result.iloc[0].sort_values(ascending=False)[:10]

奇蹟      0.471969
中華民國    0.327522
椅子      0.184183
兩岸      0.174496
九二      0.173952
漂流      0.161160
國民教育    0.149649
共識      0.148192
文明      0.146787
民主      0.144511
Name: 0, dtype: float64

In [129]:
# 轉換為array
year_keywords = result.iloc[0].sort_values(ascending=False)[:10].index.values
print(year_keywords)

['奇蹟' '中華民國' '椅子' '兩岸' '九二' '漂流' '國民教育' '共識' '文明' '民主']


In [137]:
# 十年裡前10名高分的關鍵詞
years_keywords = []
for i in range(10):
    year_keywords = result.iloc[i].sort_values(ascending=False)[:10].index.values
    years_keywords.append(year_keywords)
years_keywords

[array(['奇蹟', '中華民國', '椅子', '兩岸', '九二', '漂流', '國民教育', '共識', '文明', '民主'],
       dtype=object),
 array(['中華民國', '蔡英文', '長江', '有錢人', '政績', '宋楚瑜', '馬英九', '得票', '台灣人', '選民'],
       dtype=object),
 array(['台灣', '遊行', '火大', '老農', '休耕', '蘇貞昌', '津貼', '農業', '憲政', '退休'],
       dtype=object),
 array(['憲政', '關說', '王金平', '台灣', '政治', '經濟', '民進黨', '協議', '稱謂', '服貿'],
       dtype=object),
 array(['uber', '高鐵', '共識', '九二', '普亭', '俄羅斯', '原始', '司機', '服務', '下徹'],
       dtype=object),
 array(['經濟', '台灣', '服貿', '協議', '蔡英文', '政府', '貿易', '兩岸', '民進黨', '政治'],
       dtype=object),
 array(['勞工', '太陽花', '政府', '政策', '退場', '私校', '高教', '兩岸', '台獨', '放大'],
       dtype=object),
 array(['校長', '大學', '政府', '核四', '新黨', '經濟', '台灣', '校園', '選舉', '蘋果'],
       dtype=object),
 array(['兩岸', '台灣', '蔡英文', '超徵', '大老', '選舉', '獨派', '弱勢', '政府', '政治'],
       dtype=object),
 array(['滲透', '言論', '林靜儀', '政府', '台灣', '蔡英文', '介選', '美國', '約談', '轉發'],
       dtype=object)]

In [149]:
# 彙整為dataframe
yearly_10_keywords = pd.DataFrame(years_keywords)
yearly_10_keywords.columns = [f'keyword_{i}' for i in range(1,11)]
yearly_10_keywords["year"] = [f'{i}' for i in range(2011,2021)]
cols = yearly_10_keywords.columns.tolist()
yearly_10_keywords = yearly_10_keywords[cols[-1:] + cols[:-1]]
yearly_10_keywords

,year,keyword_1,keyword_2,keyword_3,keyword_4,keyword_5,keyword_6,keyword_7,keyword_8,keyword_9,keyword_10
0,2011,奇蹟,中華民國,椅子,兩岸,九二,漂流,國民教育,共識,文明,民主
1,2012,中華民國,蔡英文,長江,有錢人,政績,宋楚瑜,馬英九,得票,台灣人,選民
2,2013,台灣,遊行,火大,老農,休耕,蘇貞昌,津貼,農業,憲政,退休
3,2014,憲政,關說,王金平,台灣,政治,經濟,民進黨,協議,稱謂,服貿
4,2015,uber,高鐵,共識,九二,普亭,俄羅斯,原始,司機,服務,下徹
5,2016,經濟,台灣,服貿,協議,蔡英文,政府,貿易,兩岸,民進黨,政治
6,2017,勞工,太陽花,政府,政策,退場,私校,高教,兩岸,台獨,放大
7,2018,校長,大學,政府,核四,新黨,經濟,台灣,校園,選舉,蘋果
8,2019,兩岸,台灣,蔡英文,超徵,大老,選舉,獨派,弱勢,政府,政治
9,2020,滲透,言論,林靜儀,政府,台灣,蔡英文,介選,美國,約談,轉發


In [150]:
yearly_10_keywords.to_csv("top_10_keywords.csv", index=False)

# 十年完整資料

In [ ]:
years_content_ws = []
for i in range(2011,2021):
    year_content = sum(ws(list(df[df["year"]==str(i)]['content'])), [])
    w_string = str()
    for w in year_content:
        if w not in stopword_list:
            w_string += w + ' '
    years_content_ws.append(w_string)

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(years_content_ws)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
result = pd.DataFrame(denselist, columns=feature_names)
result.to_csv("tfidf_result.csv", index=False)

years_keywords = []
for i in range(10):
    year_keywords = result.iloc[i].sort_values(ascending=False)[:10].index.values
    years_keywords.append(year_keywords)

yearly_10_keywords = pd.DataFrame(years_keywords)
yearly_10_keywords.columns = [f'keyword_{i}' for i in range(1,11)]
yearly_10_keywords["year"] = [f'{i}' for i in range(2011,2021)]
cols = yearly_10_keywords.columns.tolist()
yearly_10_keywords = yearly_10_keywords[cols[-1:] + cols[:-1]]

yearly_10_keywords.to_csv("top_10_keywords.csv", index=False)

In [ ]:
yearly_10_keywords

# 補充

In [72]:
vectorizer = CountVectorizer()
transformer = TfidfTransformer()
X = vectorizer.fit_transform(years_content_ws)
tfidf = transformer.fit_transform(X)
weight = tfidf.toarray()

In [75]:
feature_names = vectorizer.get_feature_names()
dense = X.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)
df

,app,iphone,line,qe4,uber,voting,〇九,〇八年,一〇五,一一,...,ｉｍｆ,ｉｔａ,ｉｔａ２,ｍ型,ｏｐｅｃ,ｒｃｅｐ,ｔｉｆａ,ｔｐｐ,ｕ型,ｗｔｏ
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,1,1,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,7,0,7,0,0
4,1,0,0,0,26,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,2,2,0,0,1,0,6,0,0
6,0,0,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
7,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
print(weight.shape)
print(tfidf[0])
print(weight[0])

(10, 6286)
  (0, 6279)	0.011511429737661174
  (0, 6197)	0.008561389735869247
  (0, 6181)	0.017122779471738493
  (0, 6176)	0.006179660420155712
  (0, 6174)	0.009785766960985523
  (0, 6171)	0.030446755913003446
  (0, 6167)	0.009785766960985523
  (0, 6152)	0.006835726959193597
  (0, 6143)	0.011511429737661174
  (0, 6122)	0.011511429737661174
  (0, 6119)	0.011511429737661174
  (0, 6098)	0.0076116889782508615
  (0, 6080)	0.009785766960985523
  (0, 6074)	0.006835726959193597
  (0, 6073)	0.006835726959193597
  (0, 6072)	0.013671453918387194
  (0, 6067)	0.011511429737661174
  (0, 6042)	0.011511429737661174
  (0, 6039)	0.02935730088295657
  (0, 6033)	0.00932329795291787
  (0, 6031)	0.015223377956501723
  (0, 6000)	0.046045718950644694
  (0, 5996)	0.006835726959193597
  (0, 5964)	0.008561389735869247
  (0, 5960)	0.0076116889782508615
  :	:
  (0, 230)	0.011511429737661174
  (0, 212)	0.019571533921971047
  (0, 206)	0.011511429737661174
  (0, 200)	0.019571533921971047
  (0, 197)	0.00683572695919359

In [59]:
years_content = []
for i in range(2011,2021):
    iyearcontent = list(df[df["year"]==str(i)]['content'])[:5]
    years_content.append(iyearcontent)
years_content

[['今天是中華民國一百年元旦。這是艱辛困難的一百年，也是繁華豐盛的一百年。我們在此願與國人相互期勉，同心共創中華民國的「第五個奇蹟」：兩岸奇蹟。回首中華民國一百年，至少創造了五大奇蹟，包括：共和奇蹟、再生奇蹟、經濟奇蹟、民主奇蹟，及愈來愈凸顯的兩岸奇蹟。在喧囂的現實世界內，人們陷於眼前的擾攘裡；「不識奇蹟真面目，只緣身在奇蹟中」。值此民國百年元旦，我們願與國人一同檢視這「百年五大奇蹟」，回味中華民國的辛酸苦辣，思考中華民國的風華與信念，並為這「奇蹟之國」寄以勉勵與祝福。一九一二年，中華民國成為亞洲第一個民主共和國；這個「共和奇蹟」的最大成就，是創造了三民主義「民族╱民權╱民生」及「天下為公」的立國精神資產。這使得百年來經歷飄搖離亂的中華民國，今日仍能在台灣及兩岸間操持了正確的文明標竿，而孫中山先生的理想亦仍是兩岸的共同語言及共同追求。一九四九年，中華人民共和國統有大陸，中華民國政府則播遷到台灣；若非金門之役驚險獲勝，且韓戰改變了世局經緯，中華民國極可能已在國際棄絕下覆亡，台灣亦早已併入中華人民共和國。然而，一九四九年以來，世人卻見證了中華民國的「再生奇蹟」。回首前塵，中華民國政府在冷戰中的國際槓桿操作大致正確，與中共政權的對抗如八二三砲戰等亦稱成功，且對內的社經建設也能循序漸進；這使得台灣沒有併入中華人民共和國，因而也沒有經歷三面紅旗、文化大革命等浩劫，且中華民國在六十餘年後能有今日舉世共睹的政經成就，這自是「再生奇蹟」。「再生奇蹟」中的凸顯者是「經濟奇蹟」。台灣幾無自然資源，但自五○代出口蔗糖、樟腦，至當今廿一世紀成為世界資訊產業重鎮，始終在經濟建設上表現不凡。而且，此種經濟建設在近年Ｍ型社會浮現前，經由教育普及及分配管理等，相當均允地回饋給庶民百姓。中華民國的經濟奇蹟曾是世界的範本。一九八八年解嚴後的「民主奇蹟」，則是中華民國在政經表現上的再升級，亦是綜合國力的再提升。其實，中華民國的民主建設，由地方自治始，一直在進行「分期付款式」的民主實踐；正如社經建設，亦是由三七五減租始，循序進階躐等，且以社經建設成果回饋並支撐民主建設。台灣是一個經歷二二八事件及白色恐怖統治的社會，且又統獨分歧、族群多元；但中華民國如今則是由里長至總統皆公民直選的國家，而國家社會在兩岸競合的重大壓力下，在民主政治的喧囂裡，及在政黨輪替的轉換中，迄今仍能維持正確及進取的大方向，自然可稱「民